In [1]:
#https://www.youtube.com/watch?v=Qs_y0lTJAp0
import chromadb

In [3]:
chroma_client = chromadb.Client()

In [4]:
collection = chroma_client.get_or_create_collection(name = "documents")


In [5]:
collection.add(
    documents = [
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1","id2"]
)

In [6]:
results = collection.query(
    query_texts=["oranges"],
    n_results=2,
    #where_document = {"$contains":"pineapple"}

)
print(results)

#where_document is a filter, and I should check to see what they all are

{'ids': [['id2', 'id1']], 'distances': [[0.6633476614952087, 1.5742840766906738]], 'metadatas': [[None, None]], 'embeddings': None, 'documents': [['This is a document about oranges', 'This is a document about pineapple']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


In [7]:
from pprint import pprint

pprint(results)
#Distances, idea being that the smaller the distance the more similar to the embedding. 
#n_results will prioritize getting the docs with the closest distance. 

{'data': None,
 'distances': [[0.6633476614952087, 1.5742840766906738]],
 'documents': [['This is a document about oranges',
                'This is a document about pineapple']],
 'embeddings': None,
 'ids': [['id2', 'id1']],
 'included': ['metadatas', 'documents', 'distances'],
 'metadatas': [[None, None]],
 'uris': None}


In [1]:
# Ultralytics YOLO 🚀, AGPL-3.0 license
# VisDrone2019-DET dataset https://github.com/VisDrone/VisDrone-Dataset by Tianjin University
# Documentation: https://docs.ultralytics.com/datasets/detect/visdrone/
# Example usage: yolo train data=VisDrone.yaml
# parent
# ├── ultralytics
# └── datasets
#     └── VisDrone  ← downloads here (2.3 GB)

# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
"""
path: ../datasets/VisDrone # dataset root dir
train: VisDrone2019-DET-train/images # train images (relative to 'path')  6471 images
val: VisDrone2019-DET-val/images # val images (relative to 'path')  548 images
test: VisDrone2019-DET-test-dev/images # test images (optional)  1610 images

# Classes
names:
  0: pedestrian
  1: people
  2: bicycle
  3: car
  4: van
  5: truck
  6: tricycle
  7: awning-tricycle
  8: bus
  9: motor
"""

yaml = {
    "path": "./datasets/VisDrone"
}
# Download script/URL (optional) ---------------------------------------------------------------------------------------
#download: |
import os
from pathlib import Path

from ultralytics.utils.downloads import download

def visdrone2yolo(dir):
    from PIL import Image
    from tqdm import tqdm

    def convert_box(size, box):
        # Convert VisDrone box to YOLO xywh box
        dw = 1. / size[0]
        dh = 1. / size[1]
        return (box[0] + box[2] / 2) * dw, (box[1] + box[3] / 2) * dh, box[2] * dw, box[3] * dh

    (dir / 'labels').mkdir(parents=True, exist_ok=True)  # make labels directory
    pbar = tqdm((dir / 'annotations').glob('*.txt'), desc=f'Converting {dir}')
    for f in pbar:
        img_size = Image.open((dir / 'images' / f.name).with_suffix('.jpg')).size
        lines = []
        with open(f, 'r') as file:  # read annotation.txt
            for row in [x.split(',') for x in file.read().strip().splitlines()]:
                if row[4] == '0':  # VisDrone 'ignored regions' class 0
                    continue
                cls = int(row[5]) - 1
                box = convert_box(img_size, tuple(map(int, row[:4])))
                lines.append(f"{cls} {' '.join(f'{x:.6f}' for x in box)}\n")
                with open(str(f).replace(f'{os.sep}annotations{os.sep}', f'{os.sep}labels{os.sep}'), 'w') as fl:
                    fl.writelines(lines)  # write label.txt


# Download
dir = Path(yaml['path'])  # dataset root dir
urls = ['https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-val.zip',
        'https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-test-dev.zip',
        'https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-test-challenge.zip']
download(urls, dir=dir, curl=True, threads=3)

# Convert
for d in 'VisDrone2019-DET-train', 'VisDrone2019-DET-val', 'VisDrone2019-DET-test-dev':
    visdrone2yolo(dir / d)  # convert VisDrone annotations to YOLO labels

WARNING  Skipping datasets\VisDrone\VisDrone2019-DET-val.zip unzip as destination directory C:\Users\Banka\OneDrive\Desktop\OllamaTest\Training_Yolo\datasets\VisDrone\VisDrone2019-DET-val is not empty.
WARNING  Skipping datasets\VisDrone\VisDrone2019-DET-test-challenge.zip unzip as destination directory C:\Users\Banka\OneDrive\Desktop\OllamaTest\Training_Yolo\datasets\VisDrone\VisDrone2019-DET-test-challenge is not empty.
WARNING  Skipping datasets\VisDrone\VisDrone2019-DET-test-dev.zip unzip as destination directory C:\Users\Banka\OneDrive\Desktop\OllamaTest\Training_Yolo\datasets\VisDrone\VisDrone2019-DET-test-dev is not empty.


Converting datasets\VisDrone\VisDrone2019-DET-train: 6471it [01:25, 75.70it/s] 
Converting datasets\VisDrone\VisDrone2019-DET-val: 548it [00:08, 62.42it/s]
Converting datasets\VisDrone\VisDrone2019-DET-test-dev: 1610it [00:17, 90.84it/s] 


In [3]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # build a new model from YAML
#model = YOLO("yolov5nu.pt")  # load a pretrained model (recommended for training)


# Train the model
results = model.train(data="./Vis.yaml", epochs=10, imgsz=640)


Ultralytics 8.3.3  Python-3.12.5 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4080 Laptop GPU, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=./Vis.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train28, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

train: Scanning C:\Users\Banka\OneDrive\Desktop\OllamaTest\Training_Yolo\datasets\VisDrone\VisDrone2019-DET-train\labels... 6471 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6471/6471 [00:06<00:00, 1008.59it/s]

train: WARNING  C:\Users\Banka\OneDrive\Desktop\OllamaTest\Training_Yolo\datasets\VisDrone\VisDrone2019-DET-train\images\0000137_02220_d_0000163.jpg: 1 duplicate labels removed
train: WARNING  C:\Users\Banka\OneDrive\Desktop\OllamaTest\Training_Yolo\datasets\VisDrone\VisDrone2019-DET-train\images\0000140_00118_d_0000002.jpg: 1 duplicate labels removed
train: WARNING  C:\Users\Banka\OneDrive\Desktop\OllamaTest\Training_Yolo\datasets\VisDrone\VisDrone2019-DET-train\images\9999945_00000_d_0000114.jpg: 1 duplicate labels removed
train: WARNING  C:\Users\Banka\OneDrive\Desktop\OllamaTest\Training_Yolo\datasets\VisDrone\VisDrone2019-DET-train\images\9999987_00000_d_0000049.jpg: 1 duplicate labels removed


train: New cache created: C:\Users\Banka\OneDrive\Desktop\OllamaTest\Training_Yolo\datasets\VisDrone\VisDrone2019-DET-train\labels.cache


val: Scanning C:\Users\Banka\OneDrive\Desktop\OllamaTest\Training_Yolo\datasets\VisDrone\VisDrone2019-DET-val\labels.cache... 548 images, 0 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]


...

                   all        548      38759      0.347      0.263      0.246      0.141

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10      7.57G      1.445      1.131      0.939        261        640: 100%|██████████| 405/405 [00:41<00:00,  9.87it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  7.04it/s]


                   all        548      38759      0.356      0.267      0.254      0.146

10 epochs completed in 0.132 hours.
Optimizer stripped from runs\detect\train28\weights\last.pt, 5.4MB
Optimizer stripped from runs\detect\train28\weights\best.pt, 5.4MB

Validating runs\detect\train28\weights\best.pt...
Ultralytics 8.3.3  Python-3.12.5 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4080 Laptop GPU, 12282MiB)
YOLO11n summary (fused): 238 layers, 2,584,102 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:06<00:00,  2.62it/s]


                   all        548      38759      0.356      0.267      0.254      0.146
            pedestrian        520       8844      0.318      0.313      0.264       0.11
                people        482       5125      0.495      0.162      0.216     0.0736
               bicycle        364       1287      0.145     0.0179     0.0328     0.0126
                   car        515      14064      0.523      0.713      0.686      0.454
                   van        421       1975      0.399      0.298      0.282      0.192
                 truck        266        750      0.326      0.264      0.226      0.145
              tricycle        337       1045      0.351      0.165      0.149     0.0803
       awning-tricycle        220        532      0.207      0.118     0.0703     0.0469
                   bus        131        251      0.421      0.341      0.355      0.244
                 motor        485       4886      0.377      0.278      0.259      0.101
Speed: 0.2ms preproce

True
